In [ ]:
import random
import matplotlib.pyplot as plt
import networkx as nx
from operator import itemgetter

In [8]:
def makeRandomGraph(h, b, k, alpha):
    bT = nx.balanced_tree(b,h)
    graph = nx.DiGraph()
    for node1 in bT.nodes():
        probDic=dict()
        for node2 in bT.nodes():
            sl=nx.shortest_path_length(bT, node1, node2)
            p=b ** (-alpha * sl)
            probDic[node2]=p
        Z = sum(list(probDic.values()))
        i = 1
        while i <= k:
            myNodes=list(probDic.keys())
            myProbablities=list(probDic.values())
            w = random.choices(myNodes,myProbablities)[0]
            graph.add_edge(node1, w)
            Z =Z-probDic[w]
            probDic[w]= 0
            if Z>0:
                for v in probDic:
                    probDic[v]=probDic[v]/Z
            i=i+1
    return graph

In [9]:
def decenterlizedSearch(graph, s, t):
    currentNode = s
    pathLength = 0

    while currentNode != t:
        neighbors = list(graph.neighbors(currentNode))
        ndic = dict()
        for ne in neighbors:
            ndic[ne] = nx.shortest_path_length(graph, ne, t)
        x = sorted(ndic.items(), key=itemgetter(1))
        next_node = x[0][0]
        l1 = nx.shortest_path_length(graph, currentNode,  t)
        l2 = nx.shortest_path_length(graph,  next_node, t)
        if l2 > l1:
            return -1
        else:
            currentNode = next_node
            pathLength += 1

        return pathLength

In [10]:
def Exp(h,b, k , alphaList, pairs):
    averagePathLength = []
    successfullSearchProb = []

    for alpha in alphaList:
        graph = makeRandomGraph(h, b, k, alpha)
        pathLengths = []
        success = 0
        i=1
        while i<=pairs:
            s,t = random.sample(list(graph.nodes()), 2)
            try:
                pLen =  decenterlizedSearch(graph,s,t)
                if pLen > -1:
                    pathLengths.append(pLen)
                    success=success+1
            except nx.NetworkXNoPath:
                continue
            i=i+1
        if len(pathLengths)>0:
            avg=sum(pathLengths) / len(pathLengths)
            averagePathLength.append(avg)
            prob=success/pairs
            successfullSearchProb.append(prob)
    return averagePathLength , successfullSearchProb


In [11]:
h = 10
b= 2
k= 5
alphaList = []
a=0.1
while a<=10:
    alphaList.append(a)
    a=a+0.1
pairs = 1000
averagePathLengths, successfullSearchProbabilities = Exp(h, b, k, alphaList, pairs)

In [ ]:
plt.Figure()
plt.plot(alphaList, averagePathLengths, marker="o")
plt.figure()
plt.plot(alphaList, successfullSearchProbabilities , marker="o")
plt.show()